## Stage 1: Installing dependencies and environment setup


In [ ]:
!pip install tensorflow-gpu==2.0.0.alpha0

ERROR: Could not find a version that satisfies the requirement tensorflow-gpu==2.0.0.alpha0 (from versions: 2.8.0rc0, 2.8.0rc1, 2.8.0, 2.8.1, 2.8.2, 2.8.3, 2.8.4, 2.9.0rc0, 2.9.0rc1, 2.9.0rc2, 2.9.0, 2.9.1, 2.9.2, 2.9.3, 2.10.0rc0, 2.10.0rc1, 2.10.0rc2, 2.10.0rc3, 2.10.0, 2.10.1, 2.11.0rc0, 2.11.0rc1, 2.11.0rc2, 2.11.0, 2.12.0)
ERROR: No matching distribution found for tensorflow-gpu==2.0.0.alpha0


In [ ]:
!pip install pandas-datareader

## Stage 2: Importing project dependencies

In [ ]:
import math
import random
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas_datareader as data_reader

from tqdm import tqdm_notebook, tqdm
from collections import deque

In [ ]:
tf.__version__

'2.13.0'

## Stage 3: Building the AI Trader network

In [ ]:
class AI_Trader():

  def __init__(self, state_size, action_space=3, model_name="AITrader"): #Stay, Buy, Sell

    self.state_size = state_size
    self.action_space = action_space
    self.memory = deque(maxlen=2000)
    self.inventory = []
    self.model_name = model_name

    self.gamma = 0.95
    self.epsilon = 1.0
    self.epsilon_final = 0.01
    self.epsilon_decay = 0.995

    self.model = self.model_builder()

  def model_builder(self):

    model = tf.keras.models.Sequential()

    model.add(tf.keras.layers.Dense(units=32, activation='relu', input_dim=self.state_size))

    model.add(tf.keras.layers.Dense(units=64, activation='relu'))

    model.add(tf.keras.layers.Dense(units=128, activation='relu'))

    model.add(tf.keras.layers.Dense(units=self.action_space, activation='linear'))

    model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(lr=0.001))

    return model

  def trade(self, state):

    if random.random() <= self.epsilon:
      return random.randrange(self.action_space)

    actions = self.model.predict(state)
    return np.argmax(actions[0])


  def batch_train(self, batch_size):

    batch = []
    for i in range(len(self.memory) - batch_size + 1, len(self.memory)):
      batch.append(self.memory[i])

    for state, action, reward, next_state, done in batch:
      reward = reward
      if not done:
        reward = reward + self.gamma * np.amax(self.model.predict(next_state)[0])

      target = self.model.predict(state)
      target[0][action] = reward

      self.model.fit(state, target, epochs=1, verbose=0)

    if self.epsilon > self.epsilon_final:
      self.epsilon *= self.epsilon_decay

## Stage 4: Dataset preprocessing

### Defining helper functions

#### Sigmoid

In [ ]:
def sigmoid(x):
  return 1 / (1 + math.exp(-x))

#### Price format function

In [ ]:
def stocks_price_format(n):
  if n < 0:
    return "- $ {0:2f}".format(abs(n))
  else:
    return "$ {0:2f}".format(abs(n))

#### Dataset loader

In [ ]:

def dataset_loader(stock_name):

  #Complete the dataset loader function
  dataset = data_reader.DataReader(stock_name, data_source="yahoo")

  start_date = str(dataset.index[0]).split()[0]
  end_date = str(dataset.index[-1]).split()[0]

  close = dataset['Close']

  return close

### State creator

In [ ]:
def state_creator(data, timestep, window_size):

  starting_id = timestep - window_size + 1

  if starting_id >= 0:
    windowed_data = data[starting_id:timestep+1]
  else:
    windowed_data = - starting_id * [data[0]] + list(data[0:timestep+1])

  state = []
  for i in range(window_size - 1):
    state.append(sigmoid(windowed_data[i+1] - windowed_data[i]))

  return np.array([state])

### Loading a dataset

In [ ]:
stock_name = "AAPL"
data = dataset_loader(stock_name)

TypeError: ignored

In [ ]:
import yfinance as yf

# Define the ticker symbol and date range
ticker_symbol = 'AAPL'
start_date = '2022-01-01'
end_date = '2023-01-01'

# Create a DataFrame with the stock data
data = yf.download(ticker_symbol, start=start_date, end=end_date)

# Print the first few rows of the data
#print(data.head())
data=data['Close']

[*********************100%%**********************]  1 of 1 completed


## Stage 5: Training the AI Trader

### Setting hyper parameters

In [ ]:
window_size = 10
episodes = 1000

batch_size = 32
data_samples = len(data) - 1

### Defining the Trader model

In [ ]:
trader = AI_Trader(window_size)

In [ ]:
trader.model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                352       
                                                                 
 dense_1 (Dense)             (None, 64)                2112      
                                                                 
 dense_2 (Dense)             (None, 128)               8320      
                                                                 
 dense_3 (Dense)             (None, 3)                 387       
                                                                 
Total params: 11171 (43.64 KB)
Trainable params: 11171 (43.64 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### Training loop

In [ ]:
for episode in range(1, episodes + 1):

  print("Episode: {}/{}".format(episode, episodes))

  state = state_creator(data, 0, window_size + 1)

  total_profit = 0
  trader.inventory = []

  for t in tqdm(range(data_samples)):

    action = trader.trade(state)

    next_state = state_creator(data, t+1, window_size + 1)
    reward = 0

    if action == 1: #Buying
      trader.inventory.append(data[t])
      print("AI Trader bought: ", stocks_price_format(data[t]))

    elif action == 2 and len(trader.inventory) > 0: #Selling
      buy_price = trader.inventory.pop(0)

      reward = max(data[t] - buy_price, 0)
      total_profit += data[t] - buy_price
      print("AI Trader sold: ", stocks_price_format(data[t]), " Profit: " + stocks_price_format(data[t] - buy_price) )

    if t == data_samples - 1:
      done = True
    else:
      done = False

    trader.memory.append((state, action, reward, next_state, done))

    state = next_state

    if done:
      print("########################")
      print("TOTAL PROFIT: {}".format(total_profit))
      print("########################")

    if len(trader.memory) > batch_size:
      trader.batch_train(batch_size)

  if episode % 10 == 0:
    trader.model.save("ai_trader_{}.h5".format(episode))


Episode: 1/1000


  0%|          | 0/250 [00:00<?, ?it/s]

AI Trader bought:  $ 172.169998
AI Trader bought:  $ 172.190002
AI Trader bought:  $ 175.080002
AI Trader bought:  $ 175.529999
AI Trader bought:  $ 172.190002
AI Trader sold:  $ 173.070007  Profit: $ 0.900009
AI Trader sold:  $ 164.509995  Profit: - $ 7.680008
AI Trader bought:  $ 162.410004
AI Trader sold:  $ 159.779999  Profit: - $ 15.300003
AI Trader bought:  $ 159.690002
AI Trader bought:  $ 174.779999
AI Trader bought:  $ 174.610001
AI Trader sold:  $ 172.899994  Profit: - $ 2.630005
AI Trader bought:  $ 171.660004
AI Trader bought:  $ 176.279999
AI Trader sold:  $ 172.119995  Profit: - $ 0.070007
AI Trader sold:  $ 168.639999  Profit: $ 6.229996
AI Trader sold:  $ 172.789993  Profit: $ 13.099991
AI Trader sold:  $ 172.550003  Profit: - $ 2.229996
1/1 [==============================] - 0s 17ms/step


 13%|█▎        | 33/250 [00:11<01:16,  2.82it/s]

AI Trader sold:  $ 167.300003  Profit: - $ 7.309998
1/1 [==============================] - 0s 16ms/step


...

AI Trader bought:  $ 153.720001
1/1 [==============================] - 0s 23ms/step


 72%|███████▏  | 181/250 [15:02<07:15,  6.31s/it]

AI Trader sold:  $ 152.740005  Profit: - $ 1.789993
1/1 [==============================] - 0s 30ms/step


 72%|███████▏  | 181/250 [15:06<05:45,  5.01s/it]


KeyboardInterrupt: ignored